In [ ]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

In [ ]:
from utils.libs import igl
import numpy as np

file = os.path.join('..', 'datasets', 'box_1.0.stl')

v,f = igl.read_triangle_mesh(file)
bv, bf = igl.bounding_box(v)

(x0, y0, z0), (x1, y1, z1) = bv[0], bv[-1]


In [ ]:
from sdf.mesh import _cartesian_product 
N_STEP = 100
OFFSET = 2
dx, dy, dz = np.abs(bv[0]-bv[-1]) / N_STEP

X = np.linspace(x0-OFFSET*dx, x1+OFFSET*dx, N_STEP)
Y = np.linspace(y0-OFFSET*dy, y1+OFFSET*dy, N_STEP)
Z = np.linspace(z0-OFFSET*dz, z1+OFFSET*dz, N_STEP)
step = (X[1]-X[0], Y[1]-Y[0], Z[1]-Z[0])

P = _cartesian_product(X, Y, Z)

In [ ]:
import pandas as pd
df = pd.DataFrame(P, columns = ['X', 'Y', 'Z'])

df['sdf'], _, _ = igl.signed_distance(P, v, f, 4, return_normals=False)

In [ ]:
sdf, _, _ = igl.signed_distance(P, v, f, 4, return_normals=False)

In [ ]:
x = sdf.reshape((N_STEP, N_STEP, N_STEP))

In [ ]:
a = np.gradient(x, dx, dy, dz)
norm_gradient = np.linalg.norm(a, axis=0)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
d = norm_gradient[50,:,:]
fig, ax = plt.subplots()
plt.imshow(d, cmap='coolwarm', norm=colors.CenteredNorm())
plt.colorbar()
ax.set_title('min=%.6f\nmax=%.6f' % (np.min(d), np.max(d)))
ax.tick_params(axis='both', which='major', labelsize=6)

In [ ]:
import torch
delta = 0.001
c1 = 1.0/delta**2
c2 = -2.0/delta**2
c3 = 3.0*c1

A = torch.tensor([[-1.0, c3, c1, c1, c1, c2, c2, c2]])
print(A.shape)
P = A.T @ (A @ A.T).inverse() @ A
P = torch.eye(P.shape[-1]) - P
P.shape

In [ ]:
x = torch.randn((1000, 3))


delta_x = torch.tensor([[delta, 0, 0]]).tile((x.shape[0], 1))
delta_y = torch.tensor([[0, delta, 0]]).tile((x.shape[0], 1))
delta_z = torch.tensor([[0, 0, delta]]).tile((x.shape[0], 1))

augmented_x = torch.vstack((
    x,
    x + delta_x,
    x + delta_y,
    x + delta_z
))

y = torch.randn((augmented_x.shape[0],))
print(y.shape)

ny = y.reshape((4, 1000))
u = ny[0, :]
ux = ny[1, :]
uy = ny[2, :]
uz = ny[3, :]
H = torch.vstack((
    torch.ones((ny.shape[-1],)),
    u**2,
    ux**2,
    uy**2,
    uz**2,
    ux*u,
    uy*u,
    uz*u,
))
print(H.shape)

loss = torch.mean((A@H)**2)
print(loss)

In [ ]:
H_ = P @ H
print(H_.shape)
_mark = torch.tensor([[0., 1., 0., 0., 0., 0., 0., 0.]])
predict_u = _mark @ H_
print(H)

In [ ]:
t = torch.float64
u = torch.tensor([1.0], dtype=t)
ux = torch.tensor([1.0], dtype=t)
uy = torch.tensor([1.0], dtype=t)
uz = torch.tensor([1.0+delta], dtype=t)
h = torch.vstack((
    torch.ones((u.shape[-1],)),
    u**2,
    ux**2,
    uy**2,
    uz**2,
    ux*u,
    uy*u,
    uz*u,
))
torch.mm(A.to(t), h)
print(P.to(t) @ h)